In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pubhealthdataset/test.tsv
/kaggle/input/pubhealthdataset/train.tsv


In [18]:
import torch
print(torch.cuda.is_available())
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

True


'cuda:0'

In [19]:
from datasets import load_dataset, Dataset

health_fact = load_dataset("health_fact")

  0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
def remove_negative_label_rows(set_name):
    df_train = health_fact[set_name].to_pandas()
    df_train = df_train[df_train.label != -1]
    health_fact[set_name] = Dataset.from_pandas(df_train)
    health_fact[set_name] = health_fact[set_name].remove_columns(['__index_level_0__'])
    
remove_negative_label_rows('train')
remove_negative_label_rows('validation')
remove_negative_label_rows('test')

In [21]:
# health_fact['test'] = health_fact['test'].filter(lambda example: example["label"] != -1 )
# health_fact['validation'] = health_fact['validation'].filter(lambda example: example["label"] != -1 )
# health_fact['train'] = health_fact['train'].filter(lambda example: example["label"] != -1 )

In [22]:
health_fact

DatasetDict({
    train: Dataset({
        features: ['claim_id', 'claim', 'date_published', 'explanation', 'fact_checkers', 'main_text', 'sources', 'label', 'subjects'],
        num_rows: 9804
    })
    test: Dataset({
        features: ['claim_id', 'claim', 'date_published', 'explanation', 'fact_checkers', 'main_text', 'sources', 'label', 'subjects'],
        num_rows: 1233
    })
    validation: Dataset({
        features: ['claim_id', 'claim', 'date_published', 'explanation', 'fact_checkers', 'main_text', 'sources', 'label', 'subjects'],
        num_rows: 1214
    })
})

In [23]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_explanations = tokenizer(health_fact["train"]["explanation"], max_length=512, truncation=True, padding=True, return_tensors="pt") #.to(device)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/vocab.txt
loading file to

In [24]:
def tokenize_function(example):
    return tokenizer(example["explanation"], max_length=512, truncation=True, padding=True)

In [25]:
tokenized_datasets = health_fact.map(tokenize_function, batched=True)
tokenized_datasets

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['claim_id', 'claim', 'date_published', 'explanation', 'fact_checkers', 'main_text', 'sources', 'label', 'subjects', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9804
    })
    test: Dataset({
        features: ['claim_id', 'claim', 'date_published', 'explanation', 'fact_checkers', 'main_text', 'sources', 'label', 'subjects', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1233
    })
    validation: Dataset({
        features: ['claim_id', 'claim', 'date_published', 'explanation', 'fact_checkers', 'main_text', 'sources', 'label', 'subjects', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1214
    })
})

In [26]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [27]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [28]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=4) #.to(device)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
 

In [29]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [37]:
import evaluate

# metric = evaluate.load("f1", average="weighted") # not working? https://discuss.huggingface.co/t/combining-metrics-for-multiclass-predictions-evaluations/21792/10
f1_metric = evaluate.load("f1")

def compute_metrics (eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis = -1)
    
    results = {}
    results.update(f1_metric.compute(predictions=preds, references = labels, average="micro"))
    return results

# def compute_metrics(eval_preds):

#     logits, labels = eval_preds
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

In [38]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [39]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: claim, explanation, claim_id, sources, subjects, fact_checkers, main_text, date_published. If claim, explanation, claim_id, sources, subjects, fact_checkers, main_text, date_published are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1214
  Batch size = 8


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


{'eval_loss': 0.7118145823478699,
 'eval_f1': 0.7224052718286654,
 'eval_runtime': 23.7524,
 'eval_samples_per_second': 51.111,
 'eval_steps_per_second': 6.399}

In [40]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: claim, explanation, claim_id, sources, subjects, fact_checkers, main_text, date_published. If claim, explanation, claim_id, sources, subjects, fact_checkers, main_text, date_published are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 9804
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3678
  Number of train

Epoch,Training Loss,Validation Loss,F1
1,0.612200,0.697932,0.751236
2,0.402200,0.915149,0.750412
3,0.233500,1.290851,0.741351


Saving model checkpoint to test-trainer/checkpoint-500
Configuration saved in test-trainer/checkpoint-500/config.json
Model weights saved in test-trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-1000
Configuration saved in test-trainer/checkpoint-1000/config.json
Model weights saved in test-trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-1000/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: claim, explanation, claim_id, sources, subjects, fact_checkers, main_text, date_published. If claim, explanation, claim_id, sources, subjects, fact_che

TrainOutput(global_step=3678, training_loss=0.42338477586390466, metrics={'train_runtime': 1743.4626, 'train_samples_per_second': 16.87, 'train_steps_per_second': 2.11, 'total_flos': 7738761324183552.0, 'train_loss': 0.42338477586390466, 'epoch': 3.0})

In [ ]:
trainer.save_model('test-trainer-with-metrics')

In [ ]:
!pip install evaluate

In [ ]:
import evaluate

f1_metric = evaluate.load("f1")
results = f1_metric.compute(predictions=[0, 1], references=[0, 1])
print(results)

In [ ]:
tokenized_datasets["validation"][0:1]
None

In [ ]:
probe = {
    'explanation': ['Fellow Twitter users suggested @FierceFemtivist indicated she was from Louisville, Kentucky, prior to deleting her account. We were unable to locate any news reports of unusual infant deaths on 8 November 2015 in that area, nor were we able to confirm that the purported incident took place in Louisville (or at all):'],
    'labels': [3]
}

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
predictions